#Task:
Calculate efficiency of the stations based on the following features:
1. Distance from nearest bus station
2. Distance from nearest train station
3. Distance to the 3 nearest divvy stations
4. Supply vs demand of the bikes at the stations

We will add some more features to calculate efficiency after performing some more EDA to see which factors would have an effect on the efficiency.

We are yet to add the calculation for supply and demand as we need some pre processing for that.

#Use case:
To calculate how efficient is the location of each of the stations. We will also use these individual efficiencies to calculate efficiency of different regions of Chicago using the clusters created earlier.

#Analysis:
1. We were able to write an algorithm to calculate efficiency by giving certain weights (importance) to each features.
2. We then scale the efficiency so that we can obtain a value between 0 & 1. We will then decide on a threshold which can determine if a station is efficient or not. (For eg. taking the threshold as 0.5, any station with efficiency greater than 0.5 would be efficient and lesser than 0.5 would be inefficient.)

We realize that the weights that we have assigned are prone to bias. We will be conducting a survey in order to eliminate this bias and find out the importance of these features with respect to the opinion of the general public.

In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from ast import literal_eval

In [2]:
# pip install geopy

In [6]:
stations = pd.read_csv("GetNearestBusAndTrainStation.csv")
stations = stations.drop(columns="Unnamed: 0")
stations

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_train_station_name,nearest_train_station_distance,nearest_bus_station_name,nearest_bus_station_distance
0,1.594050e+18,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Kedzie-Homan,0.365048,Jackson & Kedzie,0.081453
1,6.410000e+02,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Logan Square,1.857948,Armitage & Central Park,0.342105
2,1.683530e+18,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th,3.462482,87th Street & East End,0.020426
3,3.670000e+02,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",Ashland,1.213685,35th Street & Racine,0.050069
4,1.673850e+18,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Howard,2.486505,Touhy & Francisco,0.072283
...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1.715850e+18,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Pulaski,1.058657,Roosevelt & Lawndale,0.005609
1415,1.715850e+18,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",Ashland/63rd,2.124192,74th Street & Ashland,0.034132
1416,3.200000e+02,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Racine,0.447972,Harrison & Loomis,0.217218
1417,5.400000e+01,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago,0.100847,Chicago & Milwaukee (Blue Line),0.086847


In [7]:
for i in stations.index:
    stations["nearest_train_station_distance"][i] = stations["nearest_train_station_distance"][i] * 0.621
    stations["nearest_bus_station_distance"][i] = stations["nearest_bus_station_distance"][i] * 0.621
    
stations

C:\Users\Dev Mehta\AppData\Local\Temp\ipykernel_16324\2378682040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations["nearest_train_station_distance"][i] = stations["nearest_train_station_distance"][i] * 0.621
C:\Users\Dev Mehta\AppData\Local\Temp\ipykernel_16324\2378682040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations["nearest_bus_station_distance"][i] = stations["nearest_bus_station_distance"][i] * 0.621


,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_train_station_name,nearest_train_station_distance,nearest_bus_station_name,nearest_bus_station_distance
0,1.594050e+18,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Kedzie-Homan,0.226695,Jackson & Kedzie,0.050582
1,6.410000e+02,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Logan Square,1.153785,Armitage & Central Park,0.212447
2,1.683530e+18,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th,2.150201,87th Street & East End,0.012684
3,3.670000e+02,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",Ashland,0.753699,35th Street & Racine,0.031093
4,1.673850e+18,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Howard,1.544120,Touhy & Francisco,0.044888
...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1.715850e+18,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Pulaski,0.657426,Roosevelt & Lawndale,0.003483
1415,1.715850e+18,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",Ashland/63rd,1.319123,74th Street & Ashland,0.021196
1416,3.200000e+02,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Racine,0.278191,Harrison & Loomis,0.134892
1417,5.400000e+01,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago,0.062626,Chicago & Milwaukee (Blue Line),0.053932


In [8]:
stations["efficiency"] = 0
stations

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_train_station_name,nearest_train_station_distance,nearest_bus_station_name,nearest_bus_station_distance,efficiency
0,1.594050e+18,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Kedzie-Homan,0.226695,Jackson & Kedzie,0.050582,0
1,6.410000e+02,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Logan Square,1.153785,Armitage & Central Park,0.212447,0
2,1.683530e+18,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th,2.150201,87th Street & East End,0.012684,0
3,3.670000e+02,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",Ashland,0.753699,35th Street & Racine,0.031093,0
4,1.673850e+18,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Howard,1.544120,Touhy & Francisco,0.044888,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1.715850e+18,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Pulaski,0.657426,Roosevelt & Lawndale,0.003483,0
1415,1.715850e+18,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",Ashland/63rd,1.319123,74th Street & Ashland,0.021196,0
1416,3.200000e+02,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Racine,0.278191,Harrison & Loomis,0.134892,0
1417,5.400000e+01,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago,0.062626,Chicago & Milwaukee (Blue Line),0.053932,0


In [10]:
bus_weight = 0.5
train_weight = 0.3
dist_s1_weight = 0.8
dist_s2_weight = 0.5
dist_s3_weight = 0.3

def find_efficiency(station_data):
    for i in range(len(station_data)):
        efficiency = station_data.loc[i]["nearest_bus_station_distance"] * bus_weight
        efficiency = efficiency + station_data.loc[i]["nearest_train_station_distance"] * train_weight
    

        dist = []
        for j in range(len(station_data)):
            if i != j:
                dist.append(geopy.distance.geodesic(literal_eval(station_data.loc[i]["Location"]), literal_eval(station_data.loc[j]["Location"])).miles)
        dist.sort()

        efficiency = efficiency + dist[0]*dist_s1_weight
        efficiency = efficiency + dist[1]*dist_s2_weight
        efficiency = efficiency + dist[2]*dist_s3_weight

        station_data.loc[i,"efficiency"] = efficiency

In [11]:
find_efficiency(stations)

In [12]:
stations

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_train_station_name,nearest_train_station_distance,nearest_bus_station_name,nearest_bus_station_distance,efficiency
0,1.594050e+18,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Kedzie-Homan,0.226695,Jackson & Kedzie,0.050582,0.636945
1,6.410000e+02,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Logan Square,1.153785,Armitage & Central Park,0.212447,1.037167
2,1.683530e+18,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th,2.150201,87th Street & East End,0.012684,0.730491
3,3.670000e+02,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",Ashland,0.753699,35th Street & Racine,0.031093,1.060780
4,1.673850e+18,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Howard,1.544120,Touhy & Francisco,0.044888,0.571182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1.715850e+18,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Pulaski,0.657426,Roosevelt & Lawndale,0.003483,0.400917
1415,1.715850e+18,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",Ashland/63rd,1.319123,74th Street & Ashland,0.021196,0.551977
1416,3.200000e+02,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Racine,0.278191,Harrison & Loomis,0.134892,0.605246
1417,5.400000e+01,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago,0.062626,Chicago & Milwaukee (Blue Line),0.053932,0.432207


In [13]:
df_min_max_scaled = stations.copy()
column = 'efficiency'
df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())

In [14]:
df_min_max_scaled

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_train_station_name,nearest_train_station_distance,nearest_bus_station_name,nearest_bus_station_distance,efficiency
0,1.594050e+18,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Kedzie-Homan,0.226695,Jackson & Kedzie,0.050582,0.163976
1,6.410000e+02,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Logan Square,1.153785,Armitage & Central Park,0.212447,0.279250
2,1.683530e+18,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th,2.150201,87th Street & East End,0.012684,0.190920
3,3.670000e+02,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",Ashland,0.753699,35th Street & Racine,0.031093,0.286051
4,1.673850e+18,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Howard,1.544120,Touhy & Francisco,0.044888,0.145035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1.715850e+18,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Pulaski,0.657426,Roosevelt & Lawndale,0.003483,0.095994
1415,1.715850e+18,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",Ashland/63rd,1.319123,74th Street & Ashland,0.021196,0.139503
1416,3.200000e+02,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Racine,0.278191,Harrison & Loomis,0.134892,0.154846
1417,5.400000e+01,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago,0.062626,Chicago & Milwaukee (Blue Line),0.053932,0.105006


In [15]:
stations.to_csv('stations_efficiency.csv')